# VGGT Preprocessing for Nerfstudio - Direct API Usage

This notebook demonstrates how to use VGGT (Visual Geometry Grounded deep Transformer) for structure-from-motion preprocessing **using internal vggt_utils functions directly**, bypassing the ns-process-data CLI wrapper.

**VGGT** is integrated directly into nerfstudio's custom implementation via `nerfstudio/process_data/vggt_utils.py`!

## What is VGGT?

VGGT uses deep learning to estimate camera poses and depth maps directly from images, without traditional feature matching. This can be faster and more robust than COLMAP for certain scenarios.

## Direct API Approach (This Notebook)

This notebook uses the **internal Python API** for easier prototyping and debugging:
- `nerfstudio.process_data.vggt_utils.run_vggt_ba()` - VGGT with bundle adjustment
- `nerfstudio.process_data.vggt_utils.run_vggt()` - VGGT without bundle adjustment
- `nerfstudio.process_data.colmap_utils.colmap_to_json()` - Convert COLMAP to transforms.json
- `nerfstudio.process_data.process_data_utils.convert_video_to_images()` - Extract video frames

Benefits of direct API usage:
- Full control over parameters
- Easier to prototype and experiment
- Better error handling and debugging
- No subprocess overhead
- Can modify and test code changes immediately

## Output Format

The internal functions generate COLMAP-compatible format:
- `colmap/sparse/0/cameras.bin` - Camera intrinsics
- `colmap/sparse/0/images.bin` - Camera poses and 2D keypoints
- `colmap/sparse/0/points3D.bin` - 3D point cloud with tracks
- `transforms.json` - Nerfstudio format (generated from COLMAP output)

## Prerequisites

Make sure VGGT is installed:
```bash
pip install git+https://github.com/facebookresearch/vggt.git
```

In [ ]:
%load_ext autoreload
%autoreload 2

# Verify conda environment
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we're in the nerfstudio environment
if 'nerfstudio' not in sys.executable:
    print("\n⚠️  WARNING: Not running in nerfstudio conda environment!")
    print("Please activate with: conda activate nerfstudio")
else:
    print("\n✓ Running in nerfstudio environment")

from pathlib import Path
import cv2

# Import internal nerfstudio utilities
from nerfstudio.process_data import vggt_utils
from nerfstudio.process_data import colmap_utils
from nerfstudio.process_data.process_data_utils import (
    convert_video_to_images,
    CameraModel,
)

print("✓ Imports complete")

## Configuration

Set up your input video and output paths.

In [ ]:
# Configuration
base_dir = Path("/workspace/fieldwork-data/")
video_path = base_dir / "birds/2024-02-06/SplatsSD" / "C0043.MP4"

# Output configuration
output_dir = base_dir / "birds/2024-02-06/environment/C0043"
preproc_dir = output_dir / "preproc"
image_dir = preproc_dir / "images"
colmap_dir = preproc_dir / "colmap"

# Sampling configuration
frame_proportion = 0.08
min_frames = 15

print(f"Input video: {video_path}")
print(f"Output path: {output_dir}")
print(f"Images will be saved to: {image_dir}")
print(f"COLMAP output will be saved to: {colmap_dir}")

## Step 1: Extract Video Frames

First, we'll extract frames from the video using nerfstudio's built-in utilities.

In [ ]:
# Calculate number of frames to sample
video_capture = cv2.VideoCapture(str(video_path))
n_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
n_samples = int(n_frames * frame_proportion)
n_samples = max(n_samples, min_frames)
n_samples = min(n_samples, n_frames)  # Can't sample more than total frames
video_capture.release()

print(f"Total frames in video: {n_frames}")
print(f"Frames to sample: {n_samples}")

# # Extract frames from video
# image_dir.mkdir(parents=True, exist_ok=True)

# # # Check if frames already exist
# # existing_images = list(image_dir.glob("frame_*.png"))
# # if len(existing_images) > 0 and len(existing_images) == (n_samples + 1):
# #     print(f"\n✓ {len(existing_images)} frames already extracted")
# # else:
# #     print(f"\nExtracting {n_samples} frames...")
# #     summary_log, num_extracted_frames = convert_video_to_images(
# #         video_path,
# #         image_dir=image_dir,
# #         num_frames_target=n_samples,
# #         num_downscales=0,
# #         crop_factor=(0.0, 0.0, 0.0, 0.0),
# #         verbose=True,
# #         image_prefix="frame_",
# #         keep_image_dir=False,
# #     )
# #     print(f"✓ Extracted {num_extracted_frames} frames to {image_dir}")

## Step 2: Run VGGT with Bundle Adjustment

Now we'll run VGGT inference directly using the internal API. This gives us full control over parameters.

In [ ]:
# VGGT parameters
verbose = True

# Run VGGT with bundle adjustment
vggt_utils.run_vggt(
    image_dir=image_dir,
    colmap_dir=colmap_dir,
    camera_model="PINHOLE",
    scale_factor=2.5,
    verbose=verbose,
    use_global_alignment=True,
)

print("\n✓ VGGT reconstruction complete!")
print(f"COLMAP files saved to: {colmap_dir / 'sparse' / '0'}")

In [ ]:
# VGGT parameters
camera_model = CameraModel.OPENCV
verbose = True

# Bundle adjustment parameters
ba_refine_focal_length = True
ba_refine_principal_point = False
ba_refine_extra_params = False

# Track prediction parameters (adjust these for memory constraints)
max_query_pts = 2048  # Lower to 1024 or 512 if OOM
max_points_num = 512
query_frame_num = 5   # Lower to 3 if OOM
fine_tracking = True  # Set to False if OOM (significant memory reduction)

# Run VGGT with bundle adjustment
# vggt_utils.run_vggt_ba(
#     image_dir=image_dir,
#     colmap_dir=colmap_dir,
#     camera_model=camera_model,
#     verbose=verbose,
#     ba_refine_focal_length=ba_refine_focal_length,
#     ba_refine_principal_point=ba_refine_principal_point,
#     ba_refine_extra_params=ba_refine_extra_params,
#     max_query_pts=max_query_pts,
#     max_points_num=max_points_num,
#     query_frame_num=query_frame_num,
#     fine_tracking=fine_tracking,
# )

vggt_utils.run_vggt_ba(
    image_dir=image_dir,
    colmap_dir=colmap_dir,
    camera_model="OPENCV",
    use_global_alignment=True,  # Enable GA for better pose accuracy
    max_query_pts=4096,         # Number of feature points for matching
    shared_camera=False,
    verbose=True,
)

print("\n✓ VGGT reconstruction complete!")
print(f"COLMAP files saved to: {colmap_dir / 'sparse' / '0'}")

In [ ]:
## Step 3: Convert COLMAP to Nerfstudio Format

# Convert COLMAP reconstruction to transforms.json
print("Converting COLMAP to transforms.json...")

# colmap_utils.colmap_to_json(
#     recon_dir=colmap_dir / "sparse" / "0",
#     output_dir=preproc_dir,
#     # camera_model=CameraModel.PINHOLE,
# )

# Create point cloud PLY file
import torch
import json

# transforms_path = preproc_dir / "transforms.json"
# with open(transforms_path) as f:
#     transforms = json.load(f)

# applied_transform = torch.tensor(transforms["applied_transform"])

ply_filename = "sparse_pc.ply"
# colmap_utils.create_ply_from_colmap(
#     filename=ply_filename,
#     recon_dir=colmap_dir / "sparse" / "0",
#     output_dir=preproc_dir,
#     applied_transform=applied_transform,
# )

preproc_dir = splatter.config["output_path"] / "preproc"
ply_file_path = preproc_dir / ply_filename
# transforms["ply_file_path"] = ply_filename

# print(f"\n✓ Conversion complete!")
# print(f"  - transforms.json: {transforms_path}")
# print(f"  - Point cloud: {ply_file_path}")

In [ ]:
## Step 4: Visualize the Sparse Point Cloud

import pyvista as pv
from collab_splats.utils.visualization import (
    CAMERA_KWARGS,
    MESH_KWARGS,
    VIZ_KWARGS,
    visualize_splat,
)

# Load the sparse point cloud
splat = pv.PolyData(str(ply_file_path))

pcd_kwargs = MESH_KWARGS.copy()
pcd_kwargs.update(
    {
        "point_size": 2,
        "render_points_as_spheres": True,
        "ambient": 0.3,
        "diffuse": 0.8,
        "specular": 0.1,
    }
)

plotter = visualize_splat(
    mesh=splat,
    mesh_kwargs=pcd_kwargs,
    viz_kwargs=VIZ_KWARGS,
)

plotter.show()

## Step 5: Training the Model (Optional)

After preprocessing with VGGT, train your model as usual using the Splatter wrapper or ns-train directly.

In [ ]:
%load_ext autoreload
%autoreload 2

# Verify conda environment
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we're in the nerfstudio environment
if 'nerfstudio' not in sys.executable:
    print("\n⚠️  WARNING: Not running in nerfstudio conda environment!")
    print("Please activate with: conda activate nerfstudio")
else:
    print("\n✓ Running in nerfstudio environment")

from pathlib import Path
import cv2

# Import internal nerfstudio utilities
from nerfstudio.process_data import vggt_utils
from nerfstudio.process_data import colmap_utils
from nerfstudio.process_data.process_data_utils import (
    convert_video_to_images,
    CameraModel,
)

print("✓ Imports complete")

### Evaluate on bicycles dataset

In [54]:
# Option 1: Use Splatter wrapper for training
from collab_splats.wrapper import Splatter, SplatterConfig

# Configuration
config_dir = Path("/workspace/collab-splats/docs/splats/configs/")
dataset_name = "bicycle"

# Create splatter from config
splatter = Splatter.from_config_file(
    dataset=dataset_name,
    config_dir=config_dir,
    # overrides={
    #     "frame_proportion": 0.1,
    # }
)

# splatter.preprocess()

splatter.preprocess(
    sfm_tool='vggt',
    overwrite=True, 
    kwargs={
        "refine-vggt": "",
        "camera-type": "pinhole",
        "verbose": "",
        "num_downscales": 0,
        # "skip_image_processing": "",
    }  # Enable bundle adjustment
)

[21:35:08] Copying image 1 of 194...                                                           ]8;id=964896;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=580545;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py#274\274]8;;\
           Copying image 2 of 194...                                                           ]8;id=581006;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=854890;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py#274\274]8;;\
           Copying image 3 of 194...                                                           ]8;id=502774;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=230679;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py#274\274]8;;\
           Copying image 4 of 194...       

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

[21:36:09] 🎉 Done copying images with prefix 'frame_'.                                        ]8;id=11594;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=948744;file:///workspace/nerfstudio/nerfstudio/process_data/process_data_utils.py#348\348]8;;\
Using device: cuda
Using dtype: torch.bfloat16
Loading VGGT-X model: facebook/VGGT-1B
  - Using chunk_size=256 for memory efficiency
Found 194 images
Running VGGT inference...
  - Images shape: torch.Size([194, 3, 350, 518]), dtype: torch.bfloat16
  - Model dtype: torch.bfloat16


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torc

  - extrinsic shape before squeeze: (194, 3, 4)
  - intrinsic shape before squeeze: (194, 3, 3)
✓ VGGT inference complete
  - Camera poses: (194, 3, 4)


...

Total candidate image pairs found:  1828


Matching image pairs...: 100%|██████████| 15/15 [00:05<00:00,  2.99it/s]


  - Optimizing camera poses...


Pose Optimization...: 100%|██████████| 300/300 [00:09<00:00, 30.60it/s]


Figure(1000x500)
  - Using confidence threshold: 1.0070
Building pycolmap reconstruction at WxH (518x350)...
  - Calling batch_np_matrix_to_pycolmap_wo_track with 500000 3D points
  - points_xyf shape: (500000, 3)
  - Camera type: PINHOLE
  - Created reconstruction:
    - Cameras: 1
    - Images: 194
    - Points3D: 500000
Rescaling reconstruction from WxH (518x350) to original dimensions
  - Original image sizes (WxH): 4946.0x3286.0
✓ Rescaled reconstruction to original dimensions
Writing COLMAP files to /workspace/bicycle/environment/bicycle/preproc/colmap/sparse/0
✓ COLMAP reconstruction complete!
  - Cameras: 1
  - Images: 194
  - 3D points: 500000
. Saving results to transforms.json0m
[21:40:17] 🎉 🎉 🎉 All DONE 🎉 🎉 🎉                                                ]8;id=91487;file:///workspace/nerfstudio/nerfstudio/process_data/images_to_nerfstudio_dataset.py\images_to_nerfstudio_dataset.py]8;;\:]8;id=656018;file:///workspace/nerfstudio/nerfstudio/process_data/images_to_nerfstu

In [55]:
feature_kwargs = {
    # "pipeline.model.strategy": "mcmc",
    "pipeline.model.output-depth-during-training": True,
    "pipeline.model.rasterize-mode": "antialiased",
    "pipeline.model.use-scale-regularization": True,
    "pipeline.model.random-scale": 1.0,
    "pipeline.model.num-downscales": 1,
    # "pipeline.datamanager.dataparser.downscale-factor": 1,
    # "pipeline.model.collider-params": "near_plane 0.1 far_plane 3.0",
}

splatter.extract_features(
    kwargs=feature_kwargs, 
    overwrite=True
)
print("\n✓ Training complete!")

[Taichi] version 1.7.4, llvm 15.0.4, commit b4b956fd, linux, python 3.10.18
[21:40:39] Using --data alias for --data.pipeline.datamanager.data                                          ]8;id=548538;file:///workspace/nerfstudio/nerfstudio/scripts/train.py\train.py]8;;\:]8;id=450763;file:///workspace/nerfstudio/nerfstudio/scripts/train.py#241\241]8;;\
──────────────────────────────────────────────────────── Config ────────────────────────────────────────────────────────
_TrainerConfig(
    _target=<class 'nerfstudio.engine.trainer.Trainer'>,
    output_dir=PosixPath('/workspace/bicycle/environment/bicycle'),
    method_name='rade-features',
    experiment_name='',
    project_name='nerfstudio-project',
    timestamp='2025-12-05_214039',
    machine=MachineConfig(seed=42, num_devices=1, num_machines=1, machine_rank=0, dist_url='auto', device_type='cuda'),
    logging=LoggingConfig(
        relative_log_dir=PosixPath('.'),
        steps_per_log=10,
        max_buffer_size=20,
      

/workspace/collab-splats/collab_splats/utils/camera_utils.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale))
/workspace/collab-splats/collab_splats/utils/camera_utils.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale))
/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


[21:42:20] Caching / undistorting train images                                            ]8;id=571412;file:///workspace/nerfstudio/nerfstudio/data/datamanagers/full_images_datamanager.py\full_images_datamanager.py]8;;\:]8;id=439898;file:///workspace/nerfstudio/nerfstudio/data/datamanagers/full_images_datamanager.py#239\239]8;;\
Caching / undistorting train images ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:2600:0100:01
[21:42:49] Printing max of 10 lines. Set flag --logging.local-writer.max-log-size=0 to disable line        ]8;id=844962;file:///workspace/nerfstudio/nerfstudio/utils/writer.py\writer.py]8;;\:]8;id=167414;file:///workspace/nerfstudio/nerfstudio/utils/writer.py#449\449]8;;\
           wrapping.                                                                                                    
Step (% Done)       Vis Rays / Sec       Train Iter (time)    ETA (time)           
-----------------------------------------------------------------------------------


Process ForkProcess-16:
Process ForkProcess-31:
Process ForkProcess-22:
Process ForkProcess-32:
Process ForkProcess-26:
Process ForkProcess-29:
Process ForkProcess-27:
Process ForkProcess-25:
Process ForkProcess-24:
Process ForkProcess-17:
Process ForkProcess-19:
Process ForkProcess-13:
Process ForkProcess-18:
Process ForkProcess-15:
Process ForkProcess-30:
Process ForkProcess-14:
Process ForkProcess-11:
Process ForkProcess-28:
Process ForkProcess-5:
Process ForkProcess-9:
Process ForkProcess-8:
Process ForkProcess-3:
Process ForkProcess-7:
Process ForkProcess-6:
Process ForkProcess-2:
Process ForkProcess-1:
Process ForkProcess-20:
Process ForkProcess-23:
Process ForkProcess-12:
Process ForkProcess-21:
Process ForkProcess-10:
Process ForkProcess-4:
Traceback (most recent call last):
  File "/opt/conda/envs/nerfstudio/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/envs/nerfstudio/lib/python3.10/multiprocessing/process.py", line 108,

KeyboardInterrupt: 

### Normal splatting approach


In [ ]:
# Option 1: Use Splatter wrapper for training
from collab_splats.wrapper import Splatter, SplatterConfig

# Configuration
config_dir = Path("/workspace/collab-splats/docs/splats/configs/")
dataset_name = "birds_date-02062024_video-C0043"

# Create splatter from config
splatter = Splatter.from_config_file(
    dataset=dataset_name,
    config_dir=config_dir,
    overrides={
        "frame_proportion": 0.1,
    }
)

splatter.preprocess()


# splatter.preprocess(
#     sfm_tool='vggt',
#     overwrite=True, 
#     kwargs={
#         "refine-vggt": "",
#         "camera-type": "pinhole",
#     }  # Enable bundle adjustment
# )

In [ ]:
data = splatter.inspect_data()

In [ ]:


# splatter.config["preproc_data_path"] = splatter.config["output_path"] / "preproc"

feature_kwargs = {
    # "pipeline.model.strategy": "mcmc",
    "pipeline.model.output-depth-during-training": True,
    "pipeline.model.rasterize-mode": "antialiased",
    "pipeline.model.use-scale-regularization": True,
    "pipeline.model.random-scale": 1.0,
    "pipeline.model.num-downscales": 1,
    # "pipeline.datamanager.dataparser.downscale-factor": 1,
    # "pipeline.model.collider-params": "near_plane 0.1 far_plane 3.0",
}

splatter.extract_features(
    kwargs=feature_kwargs, 
    overwrite=True
)
print("\n✓ Training complete!")

## Alternative Approaches

### 1. Using CLI (ns-process-data)

You can also use VGGT via the command line wrapper:

```bash
# Process video with VGGT
ns-process-data video \
    --data /path/to/video.mp4 \
    --output-dir /path/to/output \
    --sfm-tool vggt

# Process images with VGGT
ns-process-data images \
    --data /path/to/images/ \
    --output-dir /path/to/output \
    --sfm-tool vggt
```

### 2. Using Splatter Wrapper

```python
splatter.preprocess(sfm_tool='vggt', overwrite=True, kwargs={
    "refine-vggt-ba": "",  # Enable bundle adjustment
})
```

### 3. Direct API (This Notebook) - **Recommended for Prototyping**

```python
# Extract frames
convert_video_to_images(video_path, image_dir, num_frames_target=n_samples)

# Run VGGT with BA
vggt_utils.run_vggt_ba(image_dir, colmap_dir, camera_model, verbose=True)

# Convert to transforms.json
colmap_utils.colmap_to_json(colmap_dir / "sparse" / "0", preproc_dir, camera_model)
```

**Why use the direct API?**
- Full parameter control (max_query_pts, fine_tracking, etc.)
- No subprocess overhead
- Better error messages and debugging
- Easier to modify and test code changes
- Can experiment with different VGGT settings quickly

## When to Use VGGT vs COLMAP vs hloc

**Use VGGT when:**
- You have textureless or repetitive scenes (where COLMAP struggles)
- You want faster preprocessing
- You have good lighting and clear images
- You need dense depth estimates

**Use COLMAP when:**
- You need maximum accuracy
- You have well-textured scenes
- You have challenging camera motions or occlusions
- Traditional feature matching works well

**Use hloc when:**
- You want modern deep features (SuperPoint + SuperGlue)
- You need a balance between speed and accuracy
- You want robust matching in challenging conditions

## Direct API Implementation Details

### Key Functions Used in This Notebook

**1. Frame Extraction:**
```python
from nerfstudio.process_data.process_data_utils import convert_video_to_images

convert_video_to_images(
    video_path,
    image_dir=image_dir,
    num_frames_target=n_samples,
    verbose=True,
)
```

**2. VGGT with Bundle Adjustment:**
```python
from nerfstudio.process_data import vggt_utils

vggt_utils.run_vggt_ba(
    image_dir=image_dir,
    colmap_dir=colmap_dir,
    camera_model=camera_model,
    verbose=True,
    ba_refine_focal_length=True,
    ba_refine_principal_point=False,
    max_query_pts=2048,        # Adjust for memory
    query_frame_num=5,          # Adjust for memory
    fine_tracking=True,         # Set False to reduce memory
)
```

**3. COLMAP to Nerfstudio Conversion:**
```python
from nerfstudio.process_data import colmap_utils

colmap_utils.colmap_to_json(
    recon_dir=colmap_dir / "sparse" / "0",
    output_dir=preproc_dir,
    camera_model=camera_model,
)
```

### VGGT Processing Pipeline

The `run_vggt_ba()` function internally:

1. **Loads VGGT Model** (`facebook/VGGT-1B`)
2. **Runs Inference**: Predicts camera poses and depth maps
3. **Feature Tracking**: Uses VGGSfM's `predict_tracks()` for robust tracks
4. **Builds Reconstruction**: Creates pycolmap Reconstruction from tracks
5. **Bundle Adjustment**: Refines poses and intrinsics
6. **Writes COLMAP Files**: Saves cameras.bin, images.bin, points3D.bin

### Memory Optimization

For memory-constrained GPUs (e.g., RTX 4090 24GB):
- Set `fine_tracking=False` (significant reduction)
- Reduce `max_query_pts` to 1024 or 512
- Reduce `query_frame_num` to 3
- Use `track_resolution=518` (default, already optimized)

### Implementation Files

- **VGGT utilities**: [nerfstudio/process_data/vggt_utils.py](nerfstudio/process_data/vggt_utils.py)
- **COLMAP utilities**: [nerfstudio/process_data/colmap_utils.py](nerfstudio/process_data/colmap_utils.py)
- **Processing utilities**: [nerfstudio/process_data/process_data_utils.py](nerfstudio/process_data/process_data_utils.py)